第十一組：<br>
111356001 資管碩一 陳靖汝<br>
111356004 資管碩一 唐芸飛<br>
108306037 資管大四 施瑋翔<br>


期中考要求：

(1). 程式碼:以蝦皮資料集做成的分類模型<BR>
(2). 程式註解，解釋所採取的方法<BR>
(3). 指定資料集分類完畢<BR>
(4). 大略介紹指定資料集分類後的結果與內容分析

1. 先做 shopee_item_category 的資料前處理
2. 利用 shopee_item_category 建立預測模型
3. Jambo 資料前處理
4. 利用 jambo_6 資料，產出預測的大小類別
5. 計算正確率

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
import random

import re
from collections import Counter

### 目標：利用資料前處理後的資料進行模型訓練，並且產出最後的分類
資料集：蝦皮前處理後、jambo 前處理後資料<br>
模型：貝氏分類器<br>
預期結果：產出 jambo 直播資料集類別

In [3]:
# import shopee_jieba data
sample_df = pd.read_csv('/content/gdrive/Shareddrives/文字探勘/期中考/shopee_jieba.csv')
sample_df

,Unnamed: 0,商品名稱,大類,小類,product_name,jieba_results,jieba_string
0,144211,SIROCA自動研磨咖啡機SC-A3510,家用電器,咖啡機與周邊,自動研磨咖啡機,"['自動', '研磨', '咖啡', '機']",自動 研磨 咖啡 機
1,27434,NICOH電動冷熱奶泡機廠商直送,家用電器,咖啡機與周邊,電動冷熱奶泡機廠商直送,"['電動', '冷熱奶', '泡', '機廠', '商直', '送']",電動 冷熱奶 泡 機廠 商直 送
2,27882,Panasonic國際牌-四人份全自動雙研磨美式咖啡機NC-R601廠商直送,家用電器,咖啡機與周邊,國際牌四人份全自動雙研磨美式咖啡機廠商直送,"['國際牌', '人份', '四人份', '全自動雙', '研磨', '美式', '咖啡',...",國際牌 人份 四人份 全自動雙 研磨 美式 咖啡 機廠 商直 送
3,235987,Siroca新一代自動研磨咖啡機-紅A1210,家用電器,咖啡機與周邊,新一代自動研磨咖啡機紅,"['一代', '新一代', '自動', '研磨', '咖啡', '機紅']",一代 新一代 自動 研磨 咖啡 機紅
4,64275,超值合購荷蘭公主全自動研磨美式咖啡機冰熱奶泡機249406243000,家用電器,咖啡機與周邊,超值合購荷蘭公主全自動研磨美式咖啡機冰熱奶泡機,"['超值', '合購', '荷蘭', '公主', '全自動', '研磨', '美式', '咖...",超值 合購 荷蘭 公主 全自動 研磨 美式 咖啡 機冰 熱奶 泡機
...,...,...,...,...,...,...,...
31595,253237,大件春帶財如意玉墜贈精美中國繩乙件天然緬甸硬玉A貨,時尚配件,珠寶 / 玉石,大件春帶財如意玉墜贈精美中國繩乙件天然緬甸硬玉貨,"['大件', '春帶財', '如意', '玉墜', '贈', '精美', '中國', '繩乙...",大件 春帶財 如意 玉墜 贈 精美 中國 繩乙件 天然 緬甸 硬玉 貨
31596,146030,神祈・薰衣草紫水晶白月光天然水晶手鍊手鏈手串增智慧招貴人穩定情緒,時尚配件,珠寶 / 玉石,神祈薰衣草紫水晶白月光天然水晶手鍊手鏈手串增智慧招貴人穩定情緒,"['神祈', '衣草', '薰衣草', '水晶', '紫水晶', '月光', '白月光', ...",神祈 衣草 薰衣草 水晶 紫水晶 月光 白月光 天然 水晶 手 鍊 手鏈 手串 增 智慧 招...
31597,253246,天然緬甸硬玉A貨開運招財貔貅玉墜,時尚配件,珠寶 / 玉石,天然緬甸硬玉貨開運招財貔貅玉墜,"['天然', '緬甸', '硬玉', '貨開運', '招財', '貔貅', '玉墜']",天然 緬甸 硬玉 貨開運 招財 貔貅 玉墜
31598,118467,CityDiamond引雅天然小米粒珍珠時尚眼鏡鍊眼鏡鏈,時尚配件,珠寶 / 玉石,引雅天然小米粒珍珠時尚眼鏡鍊眼鏡鏈,"['引雅', '天然', '小', '米粒', '珍珠', '時尚', '眼鏡', '鍊',...",引雅 天然 小 米粒 珍珠 時尚 眼鏡 鍊 眼鏡 鏈


### map 大以及小類 to integer
步驟一：計算大小累的長度<br>
步驟二：利用 dictionary map 到 integer<br>
步驟三：將大小類的資料以 integer 代替<br>

In [4]:
sample_list = [sample_df]

In [5]:
#計算大小類的長度
small = len(sample_df['小類'].unique())
big = len(sample_df['大類'].unique())

class my_dictionary(dict):
 
  # __init__ function
  def __init__(self):
    self = dict()
 
  # Function to add key:value
  def add(self, key, value):
    self[key] = value

dict_small_obj = my_dictionary()
dict_big_obj = my_dictionary()

for i in range(big): #key(大類名稱):value（index）
    dict_big_obj.add(sample_df['大類'].unique()[i], "%02d" %i)     

for i in range(small): #key(小類名稱):value（index）
    dict_small_obj.add(sample_df['小類'].unique()[i], "%03d" %i)

print(dict_big_obj)
print(dict_small_obj)
for dataset in sample_list: #大小類更換成數值
    dataset['小類'] = dataset['小類'].map(dict_small_obj)
    dataset['大類'] = dataset['大類'].map(dict_big_obj)

{'家用電器': '00', '美妝保養': '01', '保健': '02', '時尚配件': '03', '嬰幼兒童裝童鞋': '04', '手機平板與周邊': '05', '居家生活': '06', '愛好與收藏品': '07', '書籍及雜誌期刊': '08', '母嬰用品': '09', '美食、伴手禮': '10', '女生包包/精品': '11', '機車類': '12', '汽車類': '13', '電腦與周邊配件': '14', '相機&空拍機': '15', '影音': '16', '男生包包': '17', '戶外與運動用品': '18', '女生衣著': '19', '男生衣著': '20', '文具、美術用具': '21', '旅行相關用品/行李箱': '22', '男鞋': '23', '女鞋': '24', '寵物': '25', '手錶': '26'}
{'咖啡機與周邊': '000', '其他家電': '001', '除毛器材': '002', '電動刮鬍刀': '003', '空氣清淨機': '004', '吹風機': '005', '電動牙刷、配件': '006', '熨斗/掛燙機': '007', '吸塵器/地板保養用具': '008', '其他生活家電': '009', '黃金 / 鉑金 / K金': '010', '項鍊': '011', '純銀': '012', '耳環': '013', '戒指': '014', '手鍊 / 手環': '015', '鑽石': '016', '吊飾 / 吊墜 / 裝飾物': '017', '帽子': '018', '平板保護周邊': '019', '手機保護周邊': '020', '智慧手錶與手環': '021', '其他': '022', '穿戴裝置配件': '023', '螢幕保護貼': '024', '筆電保護膜與保護殼': '025', '充電器': '026', '手機支架': '027', '耳機/耳麥/藍牙耳機': '028', '車用收納': '029', '傳輸線/轉接頭': '030', '燈具': '031', '手機': '032', '美食/電火鍋': '033', '讀卡機': '034', 'USB車充、車載藍芽接收器': '035', '視聽線材/轉換器'

In [6]:
sample_df['category_num'] = sample_df['大類']+' '+sample_df['小類']  # 合併大小類為一個 catogpry_num
sample_df

,Unnamed: 0,商品名稱,大類,小類,product_name,jieba_results,jieba_string,category_num
0,144211,SIROCA自動研磨咖啡機SC-A3510,00,000,自動研磨咖啡機,"['自動', '研磨', '咖啡', '機']",自動 研磨 咖啡 機,00 000
1,27434,NICOH電動冷熱奶泡機廠商直送,00,000,電動冷熱奶泡機廠商直送,"['電動', '冷熱奶', '泡', '機廠', '商直', '送']",電動 冷熱奶 泡 機廠 商直 送,00 000
2,27882,Panasonic國際牌-四人份全自動雙研磨美式咖啡機NC-R601廠商直送,00,000,國際牌四人份全自動雙研磨美式咖啡機廠商直送,"['國際牌', '人份', '四人份', '全自動雙', '研磨', '美式', '咖啡',...",國際牌 人份 四人份 全自動雙 研磨 美式 咖啡 機廠 商直 送,00 000
3,235987,Siroca新一代自動研磨咖啡機-紅A1210,00,000,新一代自動研磨咖啡機紅,"['一代', '新一代', '自動', '研磨', '咖啡', '機紅']",一代 新一代 自動 研磨 咖啡 機紅,00 000
4,64275,超值合購荷蘭公主全自動研磨美式咖啡機冰熱奶泡機249406243000,00,000,超值合購荷蘭公主全自動研磨美式咖啡機冰熱奶泡機,"['超值', '合購', '荷蘭', '公主', '全自動', '研磨', '美式', '咖...",超值 合購 荷蘭 公主 全自動 研磨 美式 咖啡 機冰 熱奶 泡機,00 000
...,...,...,...,...,...,...,...,...
31595,253237,大件春帶財如意玉墜贈精美中國繩乙件天然緬甸硬玉A貨,03,631,大件春帶財如意玉墜贈精美中國繩乙件天然緬甸硬玉貨,"['大件', '春帶財', '如意', '玉墜', '贈', '精美', '中國', '繩乙...",大件 春帶財 如意 玉墜 贈 精美 中國 繩乙件 天然 緬甸 硬玉 貨,03 631
31596,146030,神祈・薰衣草紫水晶白月光天然水晶手鍊手鏈手串增智慧招貴人穩定情緒,03,631,神祈薰衣草紫水晶白月光天然水晶手鍊手鏈手串增智慧招貴人穩定情緒,"['神祈', '衣草', '薰衣草', '水晶', '紫水晶', '月光', '白月光', ...",神祈 衣草 薰衣草 水晶 紫水晶 月光 白月光 天然 水晶 手 鍊 手鏈 手串 增 智慧 招...,03 631
31597,253246,天然緬甸硬玉A貨開運招財貔貅玉墜,03,631,天然緬甸硬玉貨開運招財貔貅玉墜,"['天然', '緬甸', '硬玉', '貨開運', '招財', '貔貅', '玉墜']",天然 緬甸 硬玉 貨開運 招財 貔貅 玉墜,03 631
31598,118467,CityDiamond引雅天然小米粒珍珠時尚眼鏡鍊眼鏡鏈,03,631,引雅天然小米粒珍珠時尚眼鏡鍊眼鏡鏈,"['引雅', '天然', '小', '米粒', '珍珠', '時尚', '眼鏡', '鍊',...",引雅 天然 小 米粒 珍珠 時尚 眼鏡 鍊 眼鏡 鏈,03 631


### model 建立
步驟一：將資料向量化<br>
步驟二：測試模型準確率<br>

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [8]:
# 將輸入的斷詞結果轉為轉為list
jieba_list = sample_df['jieba_results'].tolist()
for c in range(len(jieba_list)):
  jieba_list[c] = jieba_list[c].replace('[', '')
  jieba_list[c] = jieba_list[c].replace(']', '')
  jieba_list[c] = jieba_list[c].replace('\'', '')
  jieba_list[c] = jieba_list[c].split(', ')

In [9]:
vectorizer = CountVectorizer()

vocab = []

for l in range(len(jieba_list)):
  for v in range(len(jieba_list[l])):
      vocab.append(jieba_list[l][v])

# 向量化
vectorizer.fit(vocab)

CountVectorizer()

#### 訓練不同類別（大、小、總）目標
* 確認預測方法的準確率

* 利用大類訓練模型
* 確認大類的準確率

In [10]:
X = sample_df['jieba_string'].values.astype('U')
y = sample_df['大類'].values
X = vectorizer.transform(X)

In [11]:
# 切分訓練集與測試集，測試集占資料的30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [12]:
# 使用多項貝氏分類器訓練模型
model = MultinomialNB()
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(model.score(X_test, y_test))

0.7618143459915612


* 利用小類訓練模型
* 確認小類的準確率

In [14]:
y = sample_df['小類'].values

In [15]:
# 切分訓練集與測試集，測試集占資料的30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [16]:
# 使用多項貝氏分類器訓練模型
model = MultinomialNB()
model.fit(X_train, y_train)
model.predict(X_test)
print(model.score(X_test, y_test))

0.5427215189873418


* 利用總類訓練模型
* 確認總類的準確率

In [17]:
y = sample_df['category_num'].values

In [18]:
# 切分訓練集與測試集，測試集占資料的30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [19]:
# 使用多項貝氏分類器訓練模型
model = MultinomialNB()
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(model.score(X_test, y_test))

0.534915611814346


#### Jambo 資料匯入

In [20]:
jambo_df = pd.read_csv('/content/gdrive/Shareddrives/文字探勘/期中考/jambo_6_preprocessing_jieba.csv')
jambo_df

,Unnamed: 0,channel_name,商品id,商品名稱,product_name,jieba_results,jieba_string
0,0,邦成-自倉（C倉出貨付款全開）,3102326,大蘋0420A01韓國低分子海洋膠原蛋白粉15盒*10,大蘋韓國低分子海洋膠原蛋白粉盒,"['大蘋', '韓國', '低', '分子', '海洋', '膠原', '蛋白', '粉盒']",大蘋 韓國 低 分子 海洋 膠原 蛋白 粉盒
1,1,邦成-自倉（C倉出貨付款全開）,2764951,新三郎0314A47強力磁鐵掛勾*1,新三郎強力磁鐵掛勾,"['新三郎', '強力', '磁鐵掛', '勾']",新三郎 強力 磁鐵掛 勾
2,2,邦成-自倉（C倉出貨付款全開）,3135009,舊三郎0424A26AUSLIFE澳思萊蝸牛膠原彈潤身體修護霜100ml*2,舊三郎澳思萊蝸牛膠原彈潤身體修護霜,"['舊三郎', '澳思萊蝸', '牛膠', '原彈潤', '身體', '修護霜']",舊三郎 澳思萊蝸 牛膠 原彈潤 身體 修護霜
3,3,邦成-自倉（C倉出貨付款全開）,3981214,舊三郎0718G02速樂活衣物香氛豆膠囊*1,舊三郎速樂活衣物香氛豆膠囊,"['舊三郎', '速樂活', '衣物', '香氛', '豆', '膠囊']",舊三郎 速樂活 衣物 香氛 豆 膠囊
4,4,邦成-自倉（C倉出貨付款全開）,2951584,新三郎0403A14公主派對。極速孅印加果油*4公主派對防彈奶茶*1iSense智能電子體重秤*1,新三郎公主派對極速孅印加果油公主派對防彈奶茶智能電子體重秤,"['新三郎', '公主', '派', '對極', '速', '孅', '印加', '果油',...",新三郎 公主 派 對極 速 孅 印加 果油 公主 派 對 防彈 奶茶 智能 電子 體重 秤
...,...,...,...,...,...,...,...
18747,18747,邦成-自倉（C倉出貨付款全開）,2967713,8880406A06澎沛黑木耳900ML澎沛白木耳900ML,澎沛黑木耳澎沛白木耳,"['澎沛', '黑木', '木耳', '黑木耳', '澎沛', '白木', '木耳', '白...",澎沛 黑木 木耳 黑木耳 澎沛 白木 木耳 白木耳
18748,18748,蔥媽媽直播,5313116,第013標蔥水餃,第標蔥水餃,"['第標', '蔥', '水餃']",第標 蔥 水餃
18749,18749,TR Box寶藏屋：傘的專家、居的職人,3944880,第188標迪士尼冰壩杯870ML一168,第標迪士尼冰壩杯一,"['第標', '迪士尼', '冰壩杯', '一']",第標 迪士尼 冰壩杯 一
18750,18750,邦成-自倉（C倉出貨付款全開）,5614550,8881213A02蘭蔻超未來肌因賦活露100ml*1,蘭蔻超未來肌因賦活露,"['蘭蔻', '超未來', '肌', '因賦', '活露']",蘭蔻 超未來 肌 因賦 活露


In [21]:
# 將輸入的斷詞結果轉為轉為list
jieba_list = jambo_df['jieba_results'].tolist()
for c in range(len(jieba_list)):
  jieba_list[c] = jieba_list[c].replace('[', '')
  jieba_list[c] = jieba_list[c].replace(']', '')
  jieba_list[c] = jieba_list[c].replace('\'', '')
  jieba_list[c] = jieba_list[c].split(', ')

In [23]:
vectorizer = CountVectorizer()

for l in range(len(jieba_list)):
  for v in range(len(jieba_list[l])):
      vocab.append(jieba_list[l][v])

# 向量化
vectorizer.fit(vocab)

CountVectorizer()

### 預測 jambo 商品的大類

In [29]:
# 將 integer map 回大小類名稱函式
def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]

#### 大類

In [25]:
X_train = sample_df['jieba_string'].values.astype('U')
X_test = jambo_df['jieba_string'].values
y_train = sample_df['大類'].values
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [26]:
# 使用多項貝氏分類器進行預測
model = MultinomialNB()
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(predicted)

['01' '06' '01' ... '09' '01' '18']


In [27]:
predicted_labels_big = []
for p in predicted:
    keys = get_keys_from_value(dict_big_obj, p)
    predicted_labels_big.append(keys[0])

In [30]:
jambo_df['大類'] = predicted_labels_big

#### 小類

In [32]:
X_train = sample_df['jieba_string'].values.astype('U')
X_test = jambo_df['jieba_string'].values
y_train = sample_df['小類'].values
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [33]:
# 使用多項貝氏分類器進行預測
model = MultinomialNB()
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(predicted)

['325' '097' '372' ... '428' '323' '054']


In [34]:
predicted_labels_small = []
for p in predicted:
    keys = get_keys_from_value(dict_small_obj, p)
    predicted_labels_small.append(keys[0])

In [35]:
jambo_df['小類'] = predicted_labels_small

#### 總類

In [36]:
X_train = sample_df['jieba_string'].values.astype('U')
X_test = jambo_df['jieba_string'].values
y_train = sample_df['category_num'].values
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [37]:
# 使用多項貝氏分類器進行預測
model = MultinomialNB()
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(predicted)

['02 325' '06 097' '01 372' ... '21 428' '01 323' '18 054']


In [38]:
predicted_small = []
predicted_big = []
for i in range(len(predicted)):
    predicted_big.append(predicted[i].split(' ')[0])
    predicted_small.append(predicted[i].split(' ')[1])

In [39]:
predicted_labels_small = []
predicted_labels_big = []

for p in predicted_big:
    keys = get_keys_from_value(dict_big_obj, p)
    predicted_labels_big.append(keys[0])

for p in predicted_small:
    keys = get_keys_from_value(dict_small_obj, p)
    predicted_labels_small.append(keys[0])

In [40]:
jambo_df['大類_合'] = predicted_labels_big
jambo_df['小類_合'] = predicted_labels_small
jambo_df

,Unnamed: 0,channel_name,商品id,商品名稱,product_name,jieba_results,jieba_string,大類,小類,大類_合,小類_合
0,0,邦成-自倉（C倉出貨付款全開）,3102326,大蘋0420A01韓國低分子海洋膠原蛋白粉15盒*10,大蘋韓國低分子海洋膠原蛋白粉盒,"['大蘋', '韓國', '低', '分子', '海洋', '膠原', '蛋白', '粉盒']",大蘋 韓國 低 分子 海洋 膠原 蛋白 粉盒,美妝保養,養顏美容食品,保健,養顏美容食品
1,1,邦成-自倉（C倉出貨付款全開）,2764951,新三郎0314A47強力磁鐵掛勾*1,新三郎強力磁鐵掛勾,"['新三郎', '強力', '磁鐵掛', '勾']",新三郎 強力 磁鐵掛 勾,居家生活,黏接、修補膠帶,居家生活,黏接、修補膠帶
2,2,邦成-自倉（C倉出貨付款全開）,3135009,舊三郎0424A26AUSLIFE澳思萊蝸牛膠原彈潤身體修護霜100ml*2,舊三郎澳思萊蝸牛膠原彈潤身體修護霜,"['舊三郎', '澳思萊蝸', '牛膠', '原彈潤', '身體', '修護霜']",舊三郎 澳思萊蝸 牛膠 原彈潤 身體 修護霜,美妝保養,身體去角質、磨砂膏,美妝保養,身體去角質、磨砂膏
3,3,邦成-自倉（C倉出貨付款全開）,3981214,舊三郎0718G02速樂活衣物香氛豆膠囊*1,舊三郎速樂活衣物香氛豆膠囊,"['舊三郎', '速樂活', '衣物', '香氛', '豆', '膠囊']",舊三郎 速樂活 衣物 香氛 豆 膠囊,居家生活,衣物芳香劑,居家生活,衣物芳香劑
4,4,邦成-自倉（C倉出貨付款全開）,2951584,新三郎0403A14公主派對。極速孅印加果油*4公主派對防彈奶茶*1iSense智能電子體重秤*1,新三郎公主派對極速孅印加果油公主派對防彈奶茶智能電子體重秤,"['新三郎', '公主', '派', '對極', '速', '孅', '印加', '果油',...",新三郎 公主 派 對極 速 孅 印加 果油 公主 派 對 防彈 奶茶 智能 電子 體重 秤,家用電器,拼圖教具,母嬰用品,拼圖教具
...,...,...,...,...,...,...,...,...,...,...,...
18747,18747,邦成-自倉（C倉出貨付款全開）,2967713,8880406A06澎沛黑木耳900ML澎沛白木耳900ML,澎沛黑木耳澎沛白木耳,"['澎沛', '黑木', '木耳', '黑木耳', '澎沛', '白木', '木耳', '白...",澎沛 黑木 木耳 黑木耳 澎沛 白木 木耳 白木耳,女生衣著,裙裝,女生衣著,裙裝
18748,18748,蔥媽媽直播,5313116,第013標蔥水餃,第標蔥水餃,"['第標', '蔥', '水餃']",第標 蔥 水餃,美食、伴手禮,熟食小吃,美食、伴手禮,熟食小吃
18749,18749,TR Box寶藏屋：傘的專家、居的職人,3944880,第188標迪士尼冰壩杯870ML一168,第標迪士尼冰壩杯一,"['第標', '迪士尼', '冰壩杯', '一']",第標 迪士尼 冰壩杯 一,母嬰用品,夾子、別針、大頭針、迴紋針,文具、美術用具,夾子、別針、大頭針、迴紋針
18750,18750,邦成-自倉（C倉出貨付款全開）,5614550,8881213A02蘭蔻超未來肌因賦活露100ml*1,蘭蔻超未來肌因賦活露,"['蘭蔻', '超未來', '肌', '因賦', '活露']",蘭蔻 超未來 肌 因賦 活露,美妝保養,化妝水,美妝保養,化妝水


In [42]:
#匯出資料
jambo_df.to_csv('jambo_6_predict_category_jieba.csv', encoding='utf_8_sig')